## 전처리
### pycaret 기본 결측치 처리

In [ ]:
# from pycaret.regression import *
# import pandas as pd
# import numpy as np

# data = pd.read_csv('/content/train.csv')
# data_MLM = data.drop('HLM', axis=1)
# data_MLM = setup(data = data,
#                  target = 'MLM',
#                  text_features=['SMILES'],
#                  ignore_features = ['id'],
#                  remove_outliers = True,
#                  normalize = True,
#                 #  transformation = True,
#                 #  transform_target = True,
#                 #  polynomial_features = True,
#                 #  feature_selection = True,
#                 #  pca = True, pca_components = 10,
#                  use_gpu = True,
#                  session_id = 123,
#                 #  profile = True
#                  )
# data_MLM

# dt_MLM = create_model('dt')

# # tune model
# tuned_dt_MLM = tune_model(dt_MLM, n_iter = 50, optimize = 'RMSE')

# # ensemble model
# ensemble_model(tuned_dt_MLM, choose_better = True)

# blender_soft = blend_models(compare_models(), method = 'soft', choose_better = True)

# blender_soft = blend_models(compare_models(), method = 'soft', choose_better = True)

# # train meta-model
# lightgbm = create_model('lightgbm')

# # stack models
# stacker = stack_models(compare_models(), meta_model = lightgbm)

# # default model
# print(dt_MLM)

# # tuned model
# print(tuned_dt_MLM)

# # ensemble model
# print(ensemble_model)

# # blended model
# print(blender_soft)

# # stack models
# print(stacker.final_estimator_)

In [2]:
from pycaret.regression import *
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')

# AlogP 결측치가 없는 데이터만 선택
data_no_missing = train_data[train_data['AlogP'].notnull()]

# PyCaret setup
reg_setup = setup(data_no_missing, 
                  target='AlogP', 
                  ignore_features=['id', 'SMILES', 'MLM', 'HLM'],
                  verbose=True, 
                  session_id=123)

# 상위 3개 모델 선택
top_3_models = compare_models(n_select=3)

# 상위 모델 중 가장 성능이 좋은 모델 선택
best_model = top_3_models[0]

# AlogP 결측치가 있는 데이터만 선택
data_missing = train_data[train_data['AlogP'].isnull()]

# 예측하기 위한 특성 데이터 준비
features_for_prediction = data_missing.drop(columns=['id', 'SMILES', 'MLM', 'HLM', 'AlogP'])

# 결측치 예측
predicted_AlogP = predict_model(best_model, data=features_for_prediction)['prediction_label'].values

# 원래 데이터셋에 예측값 삽입
train_data.loc[train_data['AlogP'].isnull(), 'AlogP'] = predicted_AlogP

# 결과를 CSV 파일로 저장
train_data.to_csv('train_pycaret.csv', index=False)


,Description,Value
0,Session id,123
1,Target,AlogP
2,Target type,Regression
3,Original data shape,"(3496, 11)"
4,Transformed data shape,"(3496, 7)"
5,Transformed train set shape,"(2447, 7)"
6,Transformed test set shape,"(1049, 7)"
7,Ignore features,4
8,Numeric features,6
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.2813,0.1887,0.4328,0.9259,0.1568,0.3917,0.1780
ridge,Ridge Regression,0.2813,0.1887,0.4328,0.9259,0.1568,0.3920,0.0300
lar,Least Angle Regression,0.2813,0.1887,0.4328,0.9259,0.1568,0.3917,0.0300
br,Bayesian Ridge,0.2813,0.1887,0.4328,0.9259,0.1568,0.3921,0.0300
gbr,Gradient Boosting Regressor,0.2674,0.1905,0.4341,0.9253,0.1554,0.3455,0.0570
et,Extra Trees Regressor,0.2742,0.2099,0.4557,0.9177,0.1675,0.3701,0.0780
rf,Random Forest Regressor,0.2716,0.2111,0.4571,0.9173,0.1648,0.3832,0.1010
xgboost,Extreme Gradient Boosting,0.2792,0.2131,0.4590,0.9165,0.1632,0.3802,0.0500
huber,Huber Regressor,0.2099,0.2473,0.4956,0.9028,0.1668,0.1918,0.0320
par,Passive Aggressive Regressor,0.4152,0.3124,0.5461,0.8755,0.1850,0.5552,0.0310


In [4]:
# 테스트 데이터 로드
test_data = pd.read_csv('test.csv')

# 예측하기 위한 특성 데이터 준비 (테스트 데이터셋)
features_for_test_prediction = test_data[test_data['AlogP'].isnull()].drop(columns=['id', 'SMILES', 'AlogP'])

# 테스트 데이터의 결측치 예측
predicted_AlogP_test = predict_model(best_model, data=features_for_test_prediction)['prediction_label'].values

# 테스트 데이터셋에 예측값 삽입
test_data.loc[test_data['AlogP'].isnull(), 'AlogP'] = predicted_AlogP_test

# 테스트 데이터의 결과를 CSV 파일로 저장
test_data.to_csv('test_pycaret.csv', index=False)


In [5]:
from pycaret.regression import *
import pandas as pd

# 데이터 로딩
data = pd.read_csv('train_pycaret.csv')

In [ ]:
# MLM을 위한 setup
setup_data_MLM = setup(data = data,
                       target = 'MLM',
                       text_features=['SMILES'],
                       ignore_features = ['id', 'HLM'],
                       remove_outliers = True,
                       normalize = True,
                       feature_selection = True,
                       use_gpu = True,
                       session_id = 123,
                       verbose=True
                      )

In [ ]:
# MLM 모델 학습, 튜닝, 블렌딩, 스택킹
top_3_MLM = compare_models(n_select=3)
tuned_MLM = [tune_model(model, optimize='RMSE') for model in top_3_MLM]
blended_MLM = blend_models(tuned_MLM, method='soft', optimize='RMSE')
stacked_MLM = stack_models(tuned_MLM)

In [ ]:
# check leaderboard
get_leaderboard()

In [ ]:
# check leaderboard
lb = get_leaderboard()

# select top model
best_model_MLM = lb.iloc[0]['Model']

In [ ]:
# MLM 예측값을 데이터에 추가
data['MLM_predictions'] = predict_model(best_model_MLM, data=data)['prediction_label']

In [ ]:
# HLM을 위한 setup (MLM 예측값 포함)
setup_data_HLM = setup(data=data,
                       target='HLM',
                       ignore_features=['id', 'SMILES', 'MLM'],
                       remove_outliers=True,
                       normalize=True,
                       feature_selection=True,
                       use_gpu=True,
                       session_id=123,
                       verbose=True
                       )

In [ ]:
# HLM 모델 학습, 튜닝, 블렌딩, 스택킹
top_3_HLM = compare_models(n_select=3)
tuned_HLM = [tune_model(model, optimize='RMSE') for model in top_3_HLM]
blended_HLM = blend_models(tuned_HLM, method='soft', optimize='RMSE')
stacked_HLM = stack_models(tuned_HLM)

In [ ]:
# check leaderboard
get_leaderboard()

In [ ]:
# check leaderboard
lb = get_leaderboard()

# select top model
best_model_HLM = lb.iloc[0]['Model']

## submit csv 생성

In [ ]:
test_data = pd.read_csv('test_pycaret.csv')

# MLM 예측하기 위한 특성 데이터 준비
features_for_MLM_prediction = test_data.drop(columns=['id', 'SMILES', 'MLM', 'HLM'])

# MLM 예측
predicted_MLM = predict_model(best_model_MLM, data=features_for_MLM_prediction)['prediction_label'].values
test_data['MLM'] = predicted_MLM

# HLM 예측하기 위한 특성 데이터 준비 (MLM이 추가된 데이터를 사용)
features_for_HLM_prediction = test_data.drop(columns=['id', 'SMILES', 'HLM'])

# HLM 예측
predicted_HLM = predict_model(best_model_HLM, data=features_for_HLM_prediction)['prediction_label'].values
test_data['HLM'] = predicted_HLM

# submission.csv 파일 생성
submission = test_data[['id', 'MLM', 'HLM']]
submission.set_index('id', inplace=True)
submission.to_csv('submission_pycaret_tuned.csv')